<a href="https://colab.research.google.com/github/DmitryKutsev/congrats_generator/blob/feature%236/my_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


https://colab.research.google.com/github/sberbank-ai/ru-gpts/blob/master/examples/Finetune_RuGPTs_with_HF.ipynb#scrollTo=aZ-4Kav28cH0




# **IMPORT**

In [ ]:
import pandas as pd

In [ ]:
!pip install transformers -U

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install tqdm boto3 requests regex sacremoses

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer, AutoConfig

In [ ]:
# model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', 'gpt2')   
#  # Download model and configuration from huggingface.co and cache.
# # model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', './test/saved_model/')  
# # E.g. model was saved using `save_pretrained('./test/saved_model/')`
# model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', 'gpt2', output_attentions=True)  
# # Update configuration during loading
# assert model.config.output_attentions == True
# # Loading from a TF checkpoint file instead of a PyTorch model (slower)
# config = AutoConfig.from_pretrained('bert-base-uncased')
# # AutoConfig.from_pretrained('bert-base-uncased')
# # model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', 
# #                        './tf_model/gpt_tf_checkpoint.ckpt.index', from_tf=True, config=config)

In [ ]:
!git clone https://github.com/huggingface/transformers.git

fatal: destination path 'transformers' already exists and is not an empty directory.


In [ ]:
!pip install tensorboard
# !pip install wandb; wandb login

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
!pip install awscli
!aws s3 sync --no-sign-request s3://models.dobro.ai/gpt2/ru/unfreeze_all gpt2

# **PREPARE DATA**

In [ ]:
!wget https://raw.githubusercontent.com/DmitryKutsev/congrats_generator/main/crawler/curr_congrats.txt

--2021-12-22 05:00:07--  https://raw.githubusercontent.com/DmitryKutsev/congrats_generator/main/crawler/curr_congrats.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1471875 (1.4M) [text/plain]
Saving to: ‘curr_congrats.txt.1’

curr_congrats.txt.1 100%[===================>]   1.40M  --.-KB/s    in 0.06s   

2021-12-22 05:00:07 (25.1 MB/s) - ‘curr_congrats.txt.1’ saved [1471875/1471875]



In [ ]:
!ls

curr_congrats.txt  curr_congrats.txt.1	gpt2  sample_data  transformers


In [ ]:
with open('curr_congrats.txt') as handler:
  full_text = handler.read()
full_text[:3000]

'\nTITLE\nЭдварду Радзинскому, писателю, драматургу, сценаристу\nCONTENT\n Уважаемый Эдвард Станиславович!Примите поздравления с\xa085-летием.Человек многогранного дарования и\xa0большой созидательной энергии, Вы по\xa0праву входите в\xa0число признанных\nдраматургов, мастеров современного театрального искусства. И\xa0конечно, Вас\nхорошо знают и\xa0любят как автора целого ряда просветительских проектов, чьё\nтворчество завоевало сердца самой широкой аудитории, в\xa0том числе молодёжи.Желаю доброго здоровья, благополучия и\xa0бодрости\nдуха. Владимир Путин     Статус материала Опубликован в\xa0разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/66751   Текстовая версия      \nEND\n\nTITLE\nКоллективу радиостанции «Орфей»\nCONTENT\n Дорогие друзья!Поздравляю вас со\xa0знаменательным юбилеем\xa0– 30-летием радиостанции «Орфей».Все эти годы ваш коллектив объединяет настоящих единомышленников, профессионалов, преданных высокому искусству, стремящихся внести свой вклад в\xa0сбережение луч

In [ ]:
full_text.split('END')[:3][0].replace('\n', '').split('CONTENT')

['TITLEЭдварду Радзинскому, писателю, драматургу, сценаристу',
 ' Уважаемый Эдвард Станиславович!Примите поздравления с\xa085-летием.Человек многогранного дарования и\xa0большой созидательной энергии, Вы по\xa0праву входите в\xa0число признанныхдраматургов, мастеров современного театрального искусства. И\xa0конечно, Васхорошо знают и\xa0любят как автора целого ряда просветительских проектов, чьётворчество завоевало сердца самой широкой аудитории, в\xa0том числе молодёжи.Желаю доброго здоровья, благополучия и\xa0бодростидуха. Владимир Путин     Статус материала Опубликован в\xa0разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/66751   Текстовая версия      ']

In [ ]:
titles_list = []
congrats_list = []

for letter in full_text.split('END'):
  letter = letter.replace('\n', '').replace('TITLE', '')
  # print(letter)
  # print(letter.split('CONTENT'))
  # print(letter.split('CONTENT'))
  # break
  if len(letter.split('CONTENT')) > 1:
    titles_list.append(letter.split('CONTENT')[0])
    congrats_list.append(letter.split('CONTENT')[1])

print(titles_list[:3])
print(congrats_list[:3])
print(len(titles_list))
print(len(congrats_list))

['Эдварду Радзинскому, писателю, драматургу, сценаристу', 'Коллективу радиостанции «Орфей»', 'Михаилу Ковальчуку, президенту НИЦ «Курчатовский институт»']
[' Уважаемый Эдвард Станиславович!Примите поздравления с\xa085-летием.Человек многогранного дарования и\xa0большой созидательной энергии, Вы по\xa0праву входите в\xa0число признанныхдраматургов, мастеров современного театрального искусства. И\xa0конечно, Васхорошо знают и\xa0любят как автора целого ряда просветительских проектов, чьётворчество завоевало сердца самой широкой аудитории, в\xa0том числе молодёжи.Желаю доброго здоровья, благополучия и\xa0бодростидуха. Владимир Путин     Статус материала Опубликован в\xa0разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/66751   Текстовая версия      ', ' Дорогие друзья!Поздравляю вас со\xa0знаменательным юбилеем\xa0– 30-летием радиостанции «Орфей».Все эти годы ваш коллектив объединяет настоящих единомышленников, профессионалов, преданных высокому искусству, стремящихся внести свой вкла

In [ ]:
data = {'Title': titles_list,
        'Content': congrats_list}

my_df = pd.DataFrame(data)
my_df

,Title,Content
0,"Эдварду Радзинскому, писателю, драматургу, сце...",Уважаемый Эдвард Станиславович!Примите поздра...
1,Коллективу радиостанции «Орфей»,Дорогие друзья!Поздравляю вас со знаменательн...
2,"Михаилу Ковальчуку, президенту НИЦ «Курчатовск...",Уважаемый Михаил Валентинович!Примите мои поз...
3,"Ирине Андреевой и Ивану Штылю, победителям чем...",Уважаемые Ирина Александровна и Иван Александ...
4,Работникам и ветеранам оборонно-промышленного ...,Уважаемые друзья!Поздравляю вас с Днём оружей...
...,...,...
883,Жителям Республики Мордовия,Дорогие друзья!Поздравляю вас с юбилеем – 75-...
884,Коллективу Института физических проблем им.П.Л...,Поздравляю коллектив института с 70-летием со...
885,Жителям Красноярского края,Уважаемые друзья!Поздравляю вас с 70-летием с...
886,В.А.ЛОГИНОВУ,Уважаемый Владимир Александрович!Поздравляю В...


In [ ]:
! pip install datasets transformers

     |████████████████████████████████| 306 kB 5.6 MB/s 
     |████████████████████████████████| 1.1 MB 48.4 MB/s 
     |████████████████████████████████| 132 kB 35.5 MB/s 
     |████████████████████████████████| 243 kB 38.1 MB/s 
     |████████████████████████████████| 192 kB 21.1 MB/s 
     |████████████████████████████████| 271 kB 14.0 MB/s 
     |████████████████████████████████| 160 kB 37.0 MB/s 


In [ ]:
!git clone  https://github.com/sberbank-ai/ru-gpts

Cloning into 'ru-gpts'...
remote: Enumerating objects: 683, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 683 (delta 110), reused 141 (delta 83), pack-reused 505
Receiving objects: 100% (683/683), 413.81 KiB | 5.24 MiB/s, done.
Resolving deltas: 100% (410/410), done.


In [ ]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import re
# datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

In [ ]:
my_df['Sum'] = [my_df['Title'].tolist()[i] + my_df['Content'].tolist()[i] for i in range(len(my_df))]
my_df

,Title,Content,Sum
0,"Эдварду Радзинскому, писателю, драматургу, сце...",Уважаемый Эдвард Станиславович!Примите поздра...,"Эдварду Радзинскому, писателю, драматургу, сце..."
1,Коллективу радиостанции «Орфей»,Дорогие друзья!Поздравляю вас со знаменательн...,Коллективу радиостанции «Орфей» Дорогие друзья...
2,"Михаилу Ковальчуку, президенту НИЦ «Курчатовск...",Уважаемый Михаил Валентинович!Примите мои поз...,"Михаилу Ковальчуку, президенту НИЦ «Курчатовск..."
3,"Ирине Андреевой и Ивану Штылю, победителям чем...",Уважаемые Ирина Александровна и Иван Александ...,"Ирине Андреевой и Ивану Штылю, победителям чем..."
4,Работникам и ветеранам оборонно-промышленного ...,Уважаемые друзья!Поздравляю вас с Днём оружей...,Работникам и ветеранам оборонно-промышленного ...
...,...,...,...
883,Жителям Республики Мордовия,Дорогие друзья!Поздравляю вас с юбилеем – 75-...,Жителям Республики Мордовия Дорогие друзья!Поз...
884,Коллективу Института физических проблем им.П.Л...,Поздравляю коллектив института с 70-летием со...,Коллективу Института физических проблем им.П.Л...
885,Жителям Красноярского края,Уважаемые друзья!Поздравляю вас с 70-летием с...,Жителям Красноярского края Уважаемые друзья!По...
886,В.А.ЛОГИНОВУ,Уважаемый Владимир Александрович!Поздравляю В...,В.А.ЛОГИНОВУ Уважаемый Владимир Александрович!...


In [ ]:
train_test_ratio = 0.9
df_train, df_test = train_test_split(my_df, train_size = train_test_ratio, random_state = 1)
# train_valid_ratio = 7/9
# df_train, df_valid = train_test_split(df_full_train, train_size = train_valid_ratio, random_state = 1)

In [ ]:
def build_dataset(df, dest_path):
    f = open(dest_path, 'w')
    data = ''
    summaries = df['Sum'].tolist()
    # c = 0
    # if c < 2:
    #   print(summaries)
    #   c += 1
    for summary in summaries:
        summary = str(summary).strip()
        summary = re.sub(r"\s", " ", summary)
        summary = re.sub(r"\n", "", summary)
        summary = re.sub(r"Статус материала Опубликован в разделе: Телеграммы", "", summary)
        summary = re.sub(r"Ссылка на материал:", "", summary)
        summary = re.sub(r"Текстовая версия:", "", summary)
        summary = re.sub(r"Ссылка на материал:", "", summary)
        summary = re.sub(r"([a-zA-Z]+)", "", summary)
        summary = re.sub(r"([0-9/*#@+]+)", "", summary)
        summary = re.sub(r"Текстовая версия", "", summary)
        # bos_token = '<BOS>'  Текстовая версия
        # eos_token = '<EOS>'
        # data += bos_token + ' ' + summary + ' ' + eos_token + '\n'
        data += summary
    # c = 0
    # if c < 2:
    #   print(data)
    #   c += 1
    f.write(data)

In [ ]:
!ls
!rm 'train.txt' 'test.txt'
!ls

curr_congrats.txt    gpt2     sample_data
curr_congrats.txt.1  ru-gpts  transformers
rm: cannot remove 'train.txt': No such file or directory
rm: cannot remove 'test.txt': No such file or directory
curr_congrats.txt    gpt2     sample_data
curr_congrats.txt.1  ru-gpts  transformers


In [ ]:
build_dataset(df_train, 'train.txt')
build_dataset(df_test, 'test.txt')
# build_dataset(df_valid, 'valid.txt')

In [ ]:


# tokenizer = AutoTokenizer.from_pretrained("anonymous-german-nlp/german-gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
 
train_path = 'train.txt'
test_path = 'test.txt'

In [ ]:
!rm ./gpt-letters

rm: cannot remove './gpt-letters': No such file or directory


In [ ]:
# tokenizer.add_special_tokens({'bos_token': '<BOS>', 'eos_token': '<EOS>', 'mask_token': '<MASK>'})

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


## **TRAIN MODEL**

In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

model = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")


training_args = TrainingArguments(
    output_dir="./gpt-letters", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    # num_train_epochs=3, # number of training epochs
    # per_device_train_batch_size=32, # batch size for training
    # per_device_eval_batch_size=64,  # batch size for evaluation
    num_train_epochs=5, # number of training epochs
    per_device_train_batch_size=16, # batch size for training
    per_device_eval_batch_size=24,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=150, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

In [50]:
trainer.train()

***** Running training *****
  Num examples = 989
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 310


Step,Training Loss


Step,Training Loss


Saving model checkpoint to ./gpt-letters/checkpoint-150
Configuration saved in ./gpt-letters/checkpoint-150/config.json
Model weights saved in ./gpt-letters/checkpoint-150/pytorch_model.bin
Saving model checkpoint to ./gpt-letters/checkpoint-300
Configuration saved in ./gpt-letters/checkpoint-300/config.json
Model weights saved in ./gpt-letters/checkpoint-300/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=310, training_loss=2.387713623046875, metrics={'train_runtime': 9172.6014, 'train_samples_per_second': 0.539, 'train_steps_per_second': 0.034, 'total_flos': 323022274560000.0, 'train_loss': 2.387713623046875, 'epoch': 5.0})

In [51]:
trainer.save_model()

Saving model checkpoint to ./gpt-letters
Configuration saved in ./gpt-letters/config.json
Model weights saved in ./gpt-letters/pytorch_model.bin


In [52]:
# my_model = torch.load('./gpt-letters/pytorch_model.bin')
from transformers import pipeline

pu = pipeline('text-generation',model='./gpt-letters', tokenizer=tokenizer, max_length=500)

loading configuration file ./gpt-letters/config.json
Model config GPT2Config {
  "_name_or_path": "./gpt-letters",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.14.1",
  "use_cache": true,
  "vocab_size": 50264
}

loading configuration file ./gpt-letters/config.json
Model config GPT2Config {
  "_

In [53]:
pu("Александру Сергеевичу Пушкину")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Александру Сергеевичу Пушкину, народному артисту России Уважаемый Александр Сергеевич! Примите мои поздравления с -летием и наилучшие пожелания. Талантливый композитор, Вы по праву принадлежите к плеяде выдающихся композиторов, чьи произведения по праву считаются классикой отечественной литературы. Талантливый композитор, Вы по праву принадлежите к плеяде выдающихся композиторов, чьи произведения по праву считаются классикой отечественной литературы. Талантливый композитор, Вы по праву принадлежите к плеяде выдающихся композиторов, чьи произведения по праву считаются классикой отечественной литературы. Талантливый композитор, Вы по праву принадлежите к плеяде выдающихся композиторов, чьи произведения по праву считаются классикой отечественной литературы. Талантливый композитор, Вы по праву принадлежите к плеяде выдающихся композиторов, чьи произведения по праву считаются классикой отечественной литературы. Талантливый композитор, Вы по праву принадлежите к плеяде выдающихся композитор

In [66]:
pu("Кровавым убийцам детей")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Кровавым убийцам детей и внуков.Желаю вам здоровья, благополучия и всего наилучшего. Владимир Путин       .   Текстовая версияКоллективу и ветеранам Государственного академического театра драмы имени А.П.Чехова Дорогие друзья!Поздравляю вас с -летием Государственного академического театра драмы имени А.П.Чехова.Этот яркий, самобытный, самобытный театр, созданный в  году, по праву считается одним из лучших в России. Его постановки пользуются неизменным успехом у зрителей, пользуются неизменным успехом у коллег и многочисленных поклонников. И конечно, отмечу, что сегодня театр является одним из ведущих российских театров, его репертуар пополняется новыми постановками, которые по достоинству оценят любители драматического искусства.Желаю вам новых творческих успехов и всего наилучшего. Владимир Путин       .   Текстовая версияКоллективу и ветеранам Государственного академического театра драмы имени А.П.Чехова Дорогие друзья!Поздравляю вас с -летием Государственного академического театра 

In [67]:
pu("Уважаемые осквернители святынь и насильники")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Уважаемые осквернители святынь и насильники, осквернители святынь и насильники!Поздравляю вас с праздником – Днём памяти жертв Холокоста.Этот праздник объединяет людей, которые по праву считают себя христианами, искренне любят свою Родину, своих предков, искренне стремятся к добру и справедливости.Желаю вам здоровья, благополучия и всего наилучшего. Владимир Путин       .   Текстовая версияКоллективу и ветеранам Государственного академического театра имени Евгения Вахтангова Дорогие друзья!Поздравляю вас с -летием Государственного академического театра имени Евгения Вахтангова.Этот юбилей – дань уважения выдающимся отечественным и зарубежным актёрам, которые внесли большой вклад в развитие отечественного театра, создали неповторимую атмосферу, создали неповторимую атмосферу, которая по праву считается одной из самых ярких и запоминающихся в отечественной культуре.Важно, что сегодня Государственный академический театр имени Вахтангова – это уникальное, уникальное по красоте и содержате

In [68]:
pu("Хочу сказать этому Обэме")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Хочу сказать этому Обэме, что Вы по праву принадлежите к плеяде выдающихся мастеров, которые внесли большой вклад в развитие современного искусства, в развитие современного искусства. И сегодня Вы по праву принадлежите к плеяде выдающихся мастеров, которые внесли большой вклад в развитие современного искусства, в развитие современного искусства. Желаю Вам здоровья, благополучия и всего наилучшего. Владимир Путин       .   Текстовая версияКоллективу и ветеранам Государственного академического театра имени Евгения Вахтангова Дорогие друзья!Поздравляю вас с -летием Государственного академического театра имени Евгения Вахтангова.Этот юбилей – дань уважения выдающимся деятелям отечественного театра, которые внесли большой вклад в развитие отечественного искусства, в развитие современного искусства. И сегодня театр имени Вахтангова – это яркий, многогранный, многогранный театр, который по праву считается одним из лучших в России. И конечно, он по праву считается одним из лучших в мире. И ко

In [57]:
pu("Куцеву Диме")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Куцеву Диме, победителю чемпионата мира по спортивной борьбе  года в Будапеште в соревнованиях по греко-римской борьбе в весовой категории до  кг Уважаемый Дима! Поздравляю Вас с триумфом. Вы блестяще выступили на чемпионате мира по спортивной борьбе в Будапеште, убедительно доказали, что Вы достойны звания олимпийского чемпиона. Желаю Вам новых успехов и всего наилучшего. Владимир Путин       .   Текстовая версияКоллективу и ветеранам АО «Национальная компания «АВИА» Уважаемые друзья!Поздравляю вас с -летием АО «Национальная компания «АВИА».За прошедшие годы АО «Национальная компания «АВИА» зарекомендовало себя как крупный, надёжный, динамично развивающийся холдинг, который по праву пользуется высоким авторитетом как в России, так и за её пределами.За прошедшие годы АО «Национальная компания «АВИА» зарекомендовало себя как крупный, надёжный, динамично развивающийся холдинг, который по праву пользуется высоким авторитетом как в России, так и за её пределами.За прошедшие годы АО «Нацио

In [69]:
pu("Мычалкин")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Мычалкин, заслуженный мастер спорта СССР, олимпийский чемпион, двукратный олимпийский чемпион, двукратный чемпион мира, двукратный чемпион Европы, мира и мира, Вы по праву принадлежите к числу ярких, талантливых, преданных своему призванию спортсменов. Вы по праву принадлежите к числу ярких, талантливых, преданных своему призванию спортсменов. И конечно, особо отмечу Ваш высокий профессионализм, умение работать в команде, умение находить нестандартные решения, добиваться поставленных целей. Желаю Вам здоровья, благополучия и всего наилучшего. Дмитрий Медведев       .   Текстовая версияКоллективу и ветеранам Государственного академического театра драмы имени А.П.Чехова Дорогие друзья!Поздравляю вас с -летием Государственного академического театра драмы имени А.П.Чехова.Этот яркий, самобытный, самобытный театр, созданный в  году, по праву считается одним из самых известных и востребованных в нашей стране. Его постановки пользуются неизменным успехом у зрителей, пользуются неизменным усп

In [70]:
pu("Мычалкину")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Мычалкину, победителю чемпионата мира по водным видам спорта  года в Будапеште в соревнованиях по плаванию на дистанции  метров брассом Уважаемый Михаил Сергеевич!Поздравляю Вас с триумфом.Вы вновь подтвердили свой высокий класс, завоевав титул чемпиона мира.Желаю Вам новых достижений и всего наилучшего. Владимир Путин       .   Текстовая версияКоллективу и ветеранам Государственного музея-заповедника «Царицыно» Дорогие друзья!Поздравляю вас с -летием Государственного музея-заповедника «Царицыно».В нашей стране и за рубежом Вы известны как талантливый, самобытный художник, автор уникальных произведений, которые по праву считаются подлинными шедеврами отечественной живописи.Важно, что Вы бережно храните богатейшее собрание богатейших художественных произведений, которые по праву считаются подлинными шедеврами отечественной живописи. И конечно, Вы всегда открыты для новых творческих идей и начинаний.Желаю Вам новых успехов и всего самого доброго. Владимир Путин       .   Текстовая верси

In [ ]:
from google.colab import files

In [ ]:
import os
for file in os.listdir('./gpt-letters'):
  files.download(f'./gpt-letters/{file}')

In [ ]:
import os
for file in os.listdir('./gpt-letters'):
  print(file)

In [63]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls
!cp gpt-letters/pytorch_model.bin  gdrive/MyDrive/pytorch_model.bin
# !cp 'gpt-letters/checkpoint-150'  'gdrive/MyDrive/pytorch_model.bin'

In [64]:
!ls
!cp gpt-letters/pytorch_model.bin  gdrive/MyDrive/pytorch_model.bin
# !cp 'gpt-letters/checkpoint-150'  'gdrive/MyDrive/pytorch_model.bin'

cached_lm_GPT2Tokenizer_128_test.txt	    curr_congrats.txt.1  sample_data
cached_lm_GPT2Tokenizer_128_test.txt.lock   gdrive		 test.txt
cached_lm_GPT2Tokenizer_128_train.txt	    gpt2		 train.txt
cached_lm_GPT2Tokenizer_128_train.txt.lock  gpt-letters		 transformers
curr_congrats.txt			    ru-gpts


In [ ]:
import os
for file in os.listdir('./gpt-letters'):
  if str(file) != 'pytorch_model.bin':
    files.download(f'./gpt-letters/{file}')